In [1]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY"

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [30]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ee682e3-cb8b-49f3-9a10-e7e33f9c9957-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain import hub
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser

loader = PyPDFLoader(
    "Reglamento_Práctica_Profesional_-_CD_20062023.pdf",
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

d:\Documentos\AREP\RAGProject\.venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Los objetivos de la práctica profesional son aproximarse al ejercicio profesional involucrando todos los aspectos de la formación, contribuir a que el estudiante adquiera, desarrolle y aplique competencias del campo laboral mediante una experiencia práctica, y proporcionar elementos propios del campo laboral para analizar y ajustar los currículos de los programas. Además, se busca afianzar los lazos entre la Escuela y el sector externo. Estas prácticas pueden realizarse a nivel nacional o internacional en modalidades como práctica empresarial, social, docente o investigativa.'

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer. If you don't know the answer, say "Por favor, contáctese con la línea de soporte de la universidad"
Use three sentences maximum and keep the answer as concise as possible. The answer must have the articles where the user can find more information.
Always say "¿Tienes alguna otra duda?" at the end of the answer. The answer must be in spanish.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("¿Cuáles son los requisitos para la práctica profesional?")

'Para acceder a la práctica profesional, el estudiante debe haber aprobado el 80% de los créditos académicos de su plan de estudios, haber cursado y aprobado el Seminario - taller de preparación para el ingreso al mercado laboral y contar con la aprobación de la Decanatura. Para la práctica docente, se deben cumplir los requisitos establecidos en el documento de Lineamientos para Práctica Docente. ¿Tienes alguna otra duda?'

In [ ]:
%pip install "langserve[all]"

Ejecute el servidor!

In [ ]:
from langserve import RemoteRunnable
#Conexion al servidor
remote_chain = RemoteRunnable("http://localhost:8000/rag_chain/")

Preguntas que se pueden responder leyendo el documento:

In [ ]:

remote_chain.invoke("¿Cuáles son los requisitos para la práctica profesional?")

'Para acceder al programa de prácticas profesionales, según el Artículo 11, el estudiante debe haber aprobado el 80% de los créditos académicos de su plan de estudios, cursado y aprobado el Seminario - taller de preparación para el ingreso al mercado laboral o la asignatura obligatoria correspondiente, y contar con la aprobación de la Decanatura respectiva. En caso de la práctica docente, se deben cumplir requisitos adicionales establecidos en el documento de Lineamientos para Práctica Docente. ¿Tienes alguna otra duda?'

In [33]:
remote_chain.invoke("¿Qué beneficios adicionales reciben los estudiantes por realizar una práctica internacional?")

'Los estudiantes que realizan una práctica profesional internacional reciben un descuento del ochenta por ciento (80%) sobre el valor del derecho de matrícula para ese periodo académico, según el parágrafo 2. Además, deben mantener un seguro médico de cobertura internacional durante el desarrollo de la práctica, como se indica en el punto g. Puedes encontrar más información en el Acta 478 aprobada por el Consejo Directivo el 20 de junio de 2023. ¿Tienes alguna otra duda?'

In [34]:
remote_chain.invoke("¿Cuál es el proceso de evaluación de desempeño que realiza cada organización durante la práctica profesional?")

'El proceso de evaluación de desempeño durante la práctica profesional implica que un monitor asignado evalúa al estudiante durante la práctica según las indicaciones del programa académico. Este monitor también asigna una calificación definitiva que se registra en el sistema antes del inicio del próximo período académico. Además, se realizan al menos dos visitas del monitor a la organización para recibir información sobre el desempeño del estudiante. Puedes encontrar más información en el Reglamento de Prácticas Profesionales en Pregrado, páginas 7 y 9. ¿Tienes alguna otra duda?'

In [35]:
remote_chain.invoke("¿Qué porcentaje de los créditos académicos debe aprobar un estudiante para poder solicitar el acceso al programa de prácticas?")

'Un estudiante debe aprobar el 80% de los créditos académicos de su plan de estudios para poder solicitar el acceso al programa de prácticas profesionales, según el Artículo 11 del Reglamento de Prácticas Profesionales en Pregrado. ¿Tienes alguna otra duda?'

Preguntas que no se pueden responder leyendo el documento:

In [ ]:
remote_chain.invoke("¿Cuáles son las oportunidades de empleo disponibles para los estudiantes después de finalizar su práctica profesional?")

'Lo siento, pero la información proporcionada no incluye detalles sobre las oportunidades de empleo disponibles para los estudiantes después de finalizar su práctica profesional. Por favor, contáctese con la línea de soporte de la universidad. ¿Tienes alguna otra duda?'

In [37]:
remote_chain.invoke("¿Cuáles son las oportunidades de empleo disponibles para los estudiantes después de finalizar su práctica profesional?")

'Lo siento, pero la información proporcionada no especifica las oportunidades de empleo disponibles para los estudiantes después de finalizar su práctica profesional. Por favor, contáctese con la línea de soporte de la universidad. ¿Tienes alguna otra duda?'

In [39]:
remote_chain.invoke("¿Cuáles son los criterios específicos que utilizan las empresas para evaluar las competencias técnicas de los estudiantes en cada modalidad de práctica?")

'Lo siento, los documentos proporcionados no especifican los criterios que las empresas utilizan para evaluar las competencias técnicas de los estudiantes en cada modalidad de práctica. Por favor, contáctese con la línea de soporte de la universidad para obtener esta información. ¿Tienes alguna otra duda?'